In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)
db = client['airquality']
collection = db['stations']

In [4]:
locations = {}

In [5]:
for post in collection.find():
    print(post)
    locations[post['uid']] = {'lat': post['lat'], 'lng': post['lon']}

{'_id': ObjectId('66fbe96705105b2755240a15'), 'lat': 28.710066, 'lon': 77.24622, 'uid': 10121, 'aqi': '127', 'station': {'name': 'Sonia Vihar Water Treatment Plant DJB, Delhi, Delhi, India', 'time': '2024-10-01T19:30:00+09:00'}}
{'_id': ObjectId('66fbe96805105b2755240a16'), 'lat': 28.52123, 'lon': 76.37138, 'uid': 12741, 'aqi': '151', 'station': {'name': 'MD University, Rohtak, India', 'time': '2024-10-01T19:30:00+09:00'}}
{'_id': ObjectId('66fbe96805105b2755240a17'), 'lat': 28.999264, 'lon': 77.7590354, 'uid': 12432, 'aqi': '107', 'station': {'name': 'Ganga Nagar, Meerut, India', 'time': '2024-10-01T19:30:00+09:00'}}
{'_id': ObjectId('66fbe96805105b2755240a18'), 'lat': 28.4275, 'lon': 77.1465, 'uid': 12890, 'aqi': '105', 'station': {'name': 'Teri Gram, Gurugram, India', 'time': '2024-10-01T20:30:00+09:00'}}
{'_id': ObjectId('66fbe96805105b2755240a19'), 'lat': 28.194909, 'lon': 76.862296, 'uid': 11848, 'aqi': '75', 'station': {'name': 'RIICO Ind. Area III, Bhiwadi, India', 'time': '202

In [28]:
for key in locations:
    key=str(key)
    print(key)
    pipeline = [
        {"$group": {
            "_id": "time",  # Group by 'time'
            "ids": {"$push": "$_id"},
            "count": {"$sum": 1}
        }},
        {"$match": {
            "count": {"$gt": 1}  # Find only groups with more than one document
        }}
    ]
    duplicate_groups = list(collection.aggregate(pipeline))
    print(duplicate_groups)
    for group in duplicate_groups:
        ids_to_keep = group['ids'][0]  # Keep the first document's _id
        ids_to_delete = group['ids'][1:]  # Delete the rest
        collection.delete_many({"_id": {"$in": ids_to_delete}})
    print("Removed")
    break

10121
[]
Removed


In [ ]:
# Remove Duplicates
for key in locations:
    key=str(key)
    print(key)
    s = set()
    for doc in db[key].find():
        if doc['time'] in s:
            db[key].delete_one({'_id': doc['_id']})
            print("Removed from ", key)
        else:
            s.add(doc['time'])
    
    

In [32]:
for key in locations:
    db[str(key)].create_index([('time', 1)], unique=True)
    print("Index created for ", key)

Index created for  10121
Index created for  12741
Index created for  12432
Index created for  12890
Index created for  11848
Index created for  10116
Index created for  10704
Index created for  14134
Index created for  14146
Index created for  7024
Index created for  10114
Index created for  14141
Index created for  13689
Index created for  11318
Index created for  12435
Index created for  10115
Index created for  12452
Index created for  10117
Index created for  10118
Index created for  11856
Index created for  11963
Index created for  12816
Index created for  12813
Index created for  11863
Index created for  8685
Index created for  10708
Index created for  12447
Index created for  11861
Index created for  12430
Index created for  12434
Index created for  11266
Index created for  13717
Index created for  10120
Index created for  13805
Index created for  8672
Index created for  11267
Index created for  10705
Index created for  10124
Index created for  2555
Index created for  11866
Inde

In [36]:
#Creating station collection
import json

with open('map.json', encoding='utf-8') as f:
    data = json.load(f)
db['stations'].insert_many(data['data'])
